## 2RPFS Problem (Cmax objective) - Data treatment of result files 

In [1]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import os

%matplotlib inline

In [2]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

### List files in the output folder 

In [3]:
rootfolder = os.getcwd()
file_list = glob.glob(os.path.join(rootfolder, 'output', '*.csv'), recursive=False)
file_list

['/public/doutorado_files/2RPFS_Cmax_Budget/output/separation_10 jobs.csv',
 '/public/doutorado_files/2RPFS_Cmax_Budget/output/separation_100 jobs-2.csv',
 '/public/doutorado_files/2RPFS_Cmax_Budget/output/separation_100 jobs-3.csv',
 '/public/doutorado_files/2RPFS_Cmax_Budget/output/separation_100 jobs-4.csv',
 '/public/doutorado_files/2RPFS_Cmax_Budget/output/separation_100 jobs.csv',
 '/public/doutorado_files/2RPFS_Cmax_Budget/output/separation_100 jobs_a40_RB1004005.csv',
 '/public/doutorado_files/2RPFS_Cmax_Budget/output/separation_150 jobs-2.csv',
 '/public/doutorado_files/2RPFS_Cmax_Budget/output/separation_150 jobs.csv',
 '/public/doutorado_files/2RPFS_Cmax_Budget/output/separation_20 jobs.csv',
 '/public/doutorado_files/2RPFS_Cmax_Budget/output/separation_200 jobs-2.csv',
 '/public/doutorado_files/2RPFS_Cmax_Budget/output/separation_200 jobs-3.csv',
 '/public/doutorado_files/2RPFS_Cmax_Budget/output/separation_200 jobs-4.csv',
 '/public/doutorado_files/2RPFS_Cmax_Budget/output

In [4]:
extra_file_list = glob.glob(os.path.join(rootfolder, 'output', 'extra', '*.csv'), recursive=False)
extra_file_list

['/public/doutorado_files/2RPFS_Cmax_Budget/output/extra/RB1502008_n150_results.csv',
 '/public/doutorado_files/2RPFS_Cmax_Budget/output/extra/RB2005010_n200_results.csv']

### Read all the CSV files 

In [5]:
# Alternative script to treat files with incorrect number of coluns or faulty lines
def alternative_csv_reader(filename, delimiter=',', header=0, names=None):
    with open(filename, 'r') as file:
        lines = file.readlines() 
        count = 1
        line_list = []
        num_columns = 20
        for line in lines:  # Strips the newline character 
            #print("line{}: {}".format(count, line.strip())) 
            nc = len(line.split(','))
            if 'executionId,' in line:
                #num_columns = nc
                print('Detected {0} columns in CSV file.'.format(nc))
            else:
                if 'none,' in line:
                    if nc == num_columns:
                        line_list.append(line)
                    elif nc > num_columns:  # treat strange truncated lines
                        line = line[line.rfind('none,'):]
                        nc = len(line.split(','))
                        if nc == num_columns:
                            print('WARN: truncating line {0}, for having more columns than expected.'.format(count))
                            line_list.append(line)
                        else:
                            print('WARN: Ignoring line {0}, since it has {1} columns, instead of {2}: '.format(count, nc, num_columns), line)
                    else:  # Ignore line
                        print('WARN: Ignoring line {0}: '.format(count), line)    
                elif len(line_list[-1].split(',')) < num_columns:  # current line is a continuation of the previous one
                    line_list[-1] = line_list[-1].replace('\n', '') + line
                    print('*** Treated line {0}: '.format(count), line_list[-1])
                else:  # Ignore line
                    print('WARN: Ignoring line {0}: '.format(count), line)
            count += 1
        # assert all lines have the same number of columns
        count = 1
        for line in line_list:
            nc = len(line.split(','))
            if nc != num_columns:
                print('ERROR: Line {0} has {1} columns, instead of {2}: '.format(count, nc, num_columns), line)
            count += 1
        text_data = StringIO(''.join(line_list))
        #print('line_list: ', str(line_list))
        #print('text_data: ', text_data)
        df = pd.read_csv(text_data, delimiter=delimiter, header=header, names=names)
        return df

### Process all CSV files and append all data to a single dataframe (one per solution method: Wilson, Wagner) 

In [6]:
%%time

df_wilson = pd.DataFrame()
df_wagner = pd.DataFrame()
for filename in file_list:
    print('Processing file ', filename)
    try:
        df_ = pd.read_csv(filename, delimiter=',', header=0, names=['executionId','ub_name','instance_name','alpha','n','m','budget_Gamma','cmax','permutation','time_spent','time_to_best_sol','iterations','num_visited_solutions','num_improvements','is_optimal','validated','gap','lb','cost','cmax_dp'])
    except:  # try alternative method to read csv lines
        df_ = alternative_csv_reader(filename, delimiter=',', header=0, names=['executionId','ub_name','instance_name','alpha','n','m','budget_Gamma','cmax','permutation','time_spent','time_to_best_sol','iterations','num_visited_solutions','num_improvements','is_optimal','validated','gap','lb','cost','cmax_dp'])
    if 'wagner' not in filename:  # Wilson result file
        df_wilson = df_wilson.append(df_.copy())
    else:  # Wagner result file
        df_wagner = df_wagner.append(df_.copy())

Processing file  /public/doutorado_files/2RPFS_Cmax_Budget/output/separation_10 jobs.csv
Processing file  /public/doutorado_files/2RPFS_Cmax_Budget/output/separation_100 jobs-2.csv
Processing file  /public/doutorado_files/2RPFS_Cmax_Budget/output/separation_100 jobs-3.csv
Processing file  /public/doutorado_files/2RPFS_Cmax_Budget/output/separation_100 jobs-4.csv
Processing file  /public/doutorado_files/2RPFS_Cmax_Budget/output/separation_100 jobs.csv
Processing file  /public/doutorado_files/2RPFS_Cmax_Budget/output/separation_100 jobs_a40_RB1004005.csv
Processing file  /public/doutorado_files/2RPFS_Cmax_Budget/output/separation_150 jobs-2.csv
Processing file  /public/doutorado_files/2RPFS_Cmax_Budget/output/separation_150 jobs.csv
Processing file  /public/doutorado_files/2RPFS_Cmax_Budget/output/separation_20 jobs.csv
Processing file  /public/doutorado_files/2RPFS_Cmax_Budget/output/separation_200 jobs-2.csv
Detected 20 columns in CSV file.
Detected 20 columns in CSV file.
Detected 20 

In [7]:
df_wagner

,executionId,ub_name,instance_name,alpha,n,m,budget_Gamma,cmax,permutation,time_spent,time_to_best_sol,iterations,num_visited_solutions,num_improvements,is_optimal,validated,gap,lb,cost,cmax_dp
0,none,mip_separation,RB0103001.txt,10,10,2,20 20,285.1,10 9 7 5 3 4 2 6 1 8,6.33528,6.33528,2,2,1,true,true,0,285.1,285.1,285.1
1,none,mip_separation,RB0103001.txt,10,10,2,20 40,285.1,10 9 2 4 1 7 6 5 3 8,0.109844,0.109844,4,4,3,true,true,-1.99381e-16,285.1,285.1,285.1
2,none,mip_separation,RB0103001.txt,10,10,2,20 60,285.1,10 1 2 4 3 7 9 5 6 8,0.0807145,0.0807145,3,3,2,true,true,-1.99381e-16,285.1,285.1,285.1
3,none,mip_separation,RB0103001.txt,10,10,2,20 80,285.1,10 2 7 5 9 6 3 4 1 8,0.0552768,0.0552768,3,3,2,true,true,0,285.1,285.1,285.1
4,none,mip_separation,RB0103001.txt,10,10,2,20 100,285.1,10 1 2 7 5 4 9 6 3 8,0.0566595,0.0566595,3,3,2,true,true,1.99381e-16,285.1,285.1,285.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,none,mip_separation,RB0505010.txt,50,50,2,100 20,2196,30 2 40 33 18 29 8 35 37 25 36 24 17 19 21 20...,6.62297,6.62297,11,11,10,true,true,0,2196,2196,2196
996,none,mip_separation,RB0505010.txt,50,50,2,100 40,2196,37 24 29 18 27 50 42 31 38 47 21 6 2 46 7 48 ...,1.65574,1.65574,5,5,4,true,true,0,2196,2196,2196
997,none,mip_separation,RB0505010.txt,50,50,2,100 60,2196,20 8 48 37 39 46 19 9 41 34 49 29 45 33 42 44...,1.43077,1.43077,4,4,3,true,true,0,2196,2196,2196
998,none,mip_separation,RB0505010.txt,50,50,2,100 80,2196,20 11 39 7 3 14 47 42 17 50 9 15 31 40 46 49 ...,1.29878,1.29878,4,4,3,true,true,0,2196,2196,2196


In [8]:
df_wagner['instance_name'] = df_wagner['instance_name'].str.strip()

In [9]:
# Remove 2RPFS results for instances 
#df_wagner = df_wagner[(df_wagner['instance_name'] != 'RB2005010.txt') & (df_wagner['instance_name'] != 'RB1502008.txt')]
for filename in extra_file_list:
    print('Processing file ', filename)
    try:
        df_ = pd.read_csv(filename, delimiter=';', header=0, names=['executionId','ub_name','instance_name','alpha','n','m','budget_Gamma','cmax','permutation','time_spent','time_to_best_sol','iterations','num_visited_solutions','num_improvements','is_optimal','validated','gap','lb','cost','cmax_dp','cut_count'])
    except:  # try alternative method to read csv lines
        df_ = alternative_csv_reader(filename, delimiter=';', header=0, names=['executionId','ub_name','instance_name','alpha','n','m','budget_Gamma','cmax','permutation','time_spent','time_to_best_sol','iterations','num_visited_solutions','num_improvements','is_optimal','validated','gap','lb','cost','cmax_dp','cut_count'])
    df_wagner = df_wagner.append(df_.copy())

Processing file  /public/doutorado_files/2RPFS_Cmax_Budget/output/extra/RB1502008_n150_results.csv
Processing file  /public/doutorado_files/2RPFS_Cmax_Budget/output/extra/RB2005010_n200_results.csv


In [10]:
df_wagner

,executionId,ub_name,instance_name,alpha,n,m,budget_Gamma,cmax,permutation,time_spent,...,iterations,num_visited_solutions,num_improvements,is_optimal,validated,gap,lb,cost,cmax_dp,cut_count
0,none,mip_separation,RB0103001.txt,10,10,2,20 20,285.1,10 9 7 5 3 4 2 6 1 8,6.33528,...,2,2,1,true,true,0,285.1,285.1,285.1,NaN
1,none,mip_separation,RB0103001.txt,10,10,2,20 40,285.1,10 9 2 4 1 7 6 5 3 8,0.109844,...,4,4,3,true,true,-1.99381e-16,285.1,285.1,285.1,NaN
2,none,mip_separation,RB0103001.txt,10,10,2,20 60,285.1,10 1 2 4 3 7 9 5 6 8,0.0807145,...,3,3,2,true,true,-1.99381e-16,285.1,285.1,285.1,NaN
3,none,mip_separation,RB0103001.txt,10,10,2,20 80,285.1,10 2 7 5 9 6 3 4 1 8,0.0552768,...,3,3,2,true,true,0,285.1,285.1,285.1,NaN
4,none,mip_separation,RB0103001.txt,10,10,2,20 100,285.1,10 1 2 7 5 4 9 6 3 8,0.0566595,...,3,3,2,true,true,1.99381e-16,285.1,285.1,285.1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,none,mip_separation,RB2005010.txt,50,200,2,100 20,9192,14 2 186 83 5 6 197 155 160 22 131 133 13 9 98...,1507.53,...,12,12,11,True,True,0,9192,9192,9192,11.0
22,none,mip_separation,RB2005010.txt,50,200,2,100 40,9192,14 2 186 83 5 6 197 155 160 22 131 133 13 9 98...,945.762,...,9,9,8,True,True,0,9192,9192,9192,8.0
23,none,mip_separation,RB2005010.txt,50,200,2,100 60,9192,14 2 186 83 5 6 197 155 160 22 131 133 13 9 98...,45.3002,...,5,5,4,True,True,0,9192,9192,9192,4.0
24,none,mip_separation,RB2005010.txt,50,200,2,100 80,9192,14 2 186 83 5 6 197 155 160 22 131 133 13 9 98...,105.885,...,5,5,4,True,True,0,9192,9192,9192,4.0


### Remove duplicated header rows from both dataframes 

In [11]:
def find_invalid_values(df):
    all_invalid_values = set()
    for col in df:
        if col not in ['executionId','ub_name','instance_name','budget_Gamma','permutation','is_optimal','validated']:
            # 'alpha','n','m','cmax','time_spent','time_to_best_sol','iterations','num_visited_solutions','num_improvements','gap','lb','cost','cmax_dp'
            a = pd.to_numeric(df[col], errors='coerce')
            idx = a.isna()
            invalid_values = df.loc[idx][col].unique()
            all_invalid_values.update(invalid_values)
        #elif col in ['is_optimal','validated']
    print('Invalid values:', all_invalid_values)
    return all_invalid_values

In [12]:
find_invalid_values(df_wagner)

Invalid values: {'alpha', nan, 'lb', 'time_spent', 'cmax', 'cost', 'gap', 'num_visited_solutions', 'time_to_best_sol', 'cmax_dp', 'n', 'm', 'iterations', 'num_improvements'}


{'alpha',
 'cmax',
 'cmax_dp',
 'cost',
 'gap',
 'iterations',
 'lb',
 'm',
 'n',
 nan,
 'num_improvements',
 'num_visited_solutions',
 'time_spent',
 'time_to_best_sol'}

In [13]:
def filter_invalid_values(df):
    # IMPORTANT: AVOID FILTERING 'NAN' VALUES
    for invalid_value in find_invalid_values(df):
        if isinstance(invalid_value, str):  # Evita filtrar os nan
            df = df[~(df == invalid_value).any(axis=1)]
    return df

In [14]:
df_wilson = filter_invalid_values(df_wilson)
find_invalid_values(df_wilson)

Invalid values: {'alpha', 'lb', 'time_spent', 'cmax', 'cost', 'gap', 'num_visited_solutions', 'time_to_best_sol', 'cmax_dp', 'n', 'm', 'iterations', 'num_improvements'}
Invalid values: set()


set()

In [15]:
df_wagner = filter_invalid_values(df_wagner)
find_invalid_values(df_wagner)

Invalid values: {'alpha', nan, 'lb', 'time_spent', 'cmax', 'cost', 'gap', 'num_visited_solutions', 'time_to_best_sol', 'cmax_dp', 'n', 'm', 'iterations', 'num_improvements'}
Invalid values: {nan}


{nan}

### Convert column types from object 

In [16]:
def convert_column_types(df):
    for col in df:
        if col in ['alpha','n','m','cmax','time_spent','time_to_best_sol','iterations','num_visited_solutions','num_improvements','gap','lb','cost','cmax_dp']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        elif col in ['is_optimal','validated']:
            df[col] = df[col].astype('bool')
    return df

In [17]:
%%time
df_wilson = convert_column_types(df_wilson)
df_wagner = convert_column_types(df_wagner)
df_wagner.dtypes

CPU times: user 247 ms, sys: 7.15 ms, total: 254 ms
Wall time: 488 ms


executionId               object
ub_name                   object
instance_name             object
alpha                      int64
n                          int64
m                          int64
budget_Gamma              object
cmax                     float64
permutation               object
time_spent               float64
time_to_best_sol         float64
iterations                 int64
num_visited_solutions      int64
num_improvements           int64
is_optimal                  bool
validated                   bool
gap                      float64
lb                       float64
cost                     float64
cmax_dp                  float64
cut_count                float64
dtype: object

### Trim existing string columns 

In [18]:
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

In [19]:
df_wilson = trim_all_columns(df_wilson)
df_wagner = trim_all_columns(df_wagner)

### Include a column with the name of the underlying C&CG MILP Model

In [20]:
df_wilson['model'] = 'Wilson'
df_wagner['model'] = 'Wagner'

### Concatenate dataframes 

In [21]:
df = df_wilson.append(df_wagner)
df.head(4)

,executionId,ub_name,instance_name,alpha,n,m,budget_Gamma,cmax,permutation,time_spent,...,num_visited_solutions,num_improvements,is_optimal,validated,gap,lb,cost,cmax_dp,model,cut_count
0,none,mip_separation,RB0103001.txt,10,10,2,20 20,285.1,10 1 2 3 5 6 7 9 4 8,5.479829,...,3,2,True,True,-1.993806e-16,285.1,285.1,285.1,Wilson,NaN
1,none,mip_separation,RB0103001.txt,10,10,2,20 40,285.1,1 10 7 4 2 3 6 5 9 8,0.076683,...,3,2,True,True,0.000000e+00,285.1,285.1,285.1,Wilson,NaN
2,none,mip_separation,RB0103001.txt,10,10,2,20 60,285.1,1 10 9 7 2 3 4 6 5 8,0.078870,...,3,2,True,True,0.000000e+00,285.1,285.1,285.1,Wilson,NaN
3,none,mip_separation,RB0103001.txt,10,10,2,20 80,285.1,10 1 2 9 4 5 3 7 6 8,0.078112,...,3,2,True,True,0.000000e+00,285.1,285.1,285.1,Wilson,NaN


### Split the column budget_Gamma into Gamma1 and Gamma2 

In [22]:
# new data frame with split value columns 
new = df["budget_Gamma"].str.split(" ", n = 1, expand = True) 
# making separate first name column from new data frame 
df["Gamma1"]= new[0] 
# making separate last name column from new data frame 
df["Gamma2"]= new[1] 
# convert Gamma columns to numeric
df["Gamma1"] = pd.to_numeric(df["Gamma1"], errors='coerce')
df["Gamma2"] = pd.to_numeric(df["Gamma2"], errors='coerce')
df = df.replace(np.nan, -100, regex=True)
df["Gamma1"] = df["Gamma1"].astype(int)
df["Gamma2"] = df["Gamma2"].astype(int)
df["n"] = df["n"].astype(int)
df["m"] = df["m"].astype(int)
df["alpha"] = df["alpha"].astype(int)

### Fix instance names 

The original instance names, as provided by Ying in the instance file zip, were assembled incorrectly.

The problem lies in the alpha percentage. We are now going to fix this issue.

In [23]:
df['seq'] = df['instance_name'].str[7:9]
df['n_str'] = df['n'].astype(str).str.zfill(3)
df['alpha_str'] = df['alpha'].astype(str)

In [24]:
df['instance_name'] = 'RB' + df['n_str'] + df['alpha_str'] + df['seq'] + '.txt'
df.drop(columns=['n_str', 'alpha_str'], inplace=True)

### Filter paper instance results (n=150 and n=200)

In [25]:
len(df.index)

15416

In [26]:
df[(df['cut_count'] < 0)&(df['instance_name'] == 'RB2005010.txt')]

,executionId,ub_name,instance_name,alpha,n,m,budget_Gamma,cmax,permutation,time_spent,...,validated,gap,lb,cost,cmax_dp,model,cut_count,Gamma1,Gamma2,seq
90,none,mip_separation,RB2005010.txt,50,200,2,20 20,7205.5,73 27 150 60 83 96 113 177 184 48 158 148 109 ...,7600.929033,...,True,0.025840,7024.0,7205.5,7205.5,Wilson,-100.0,20,20,10
91,none,mip_separation,RB2005010.txt,50,200,2,20 40,7743.0,73 27 150 60 83 96 113 177 184 48 158 148 109 ...,7452.786222,...,True,0.002006,7727.5,7743.0,7743.0,Wilson,-100.0,20,40,10
92,none,mip_separation,RB2005010.txt,50,200,2,20 60,8350.0,73 27 150 60 83 96 113 177 184 48 158 148 109 ...,8187.741883,...,True,0.001860,8334.5,8350.0,8350.0,Wilson,-100.0,20,60,10
93,none,mip_separation,RB2005010.txt,50,200,2,20 80,8764.5,73 27 150 60 83 96 113 177 184 48 158 148 109 ...,7951.646845,...,True,0.001772,8749.0,8764.5,8764.5,Wilson,-100.0,20,80,10
94,none,mip_separation,RB2005010.txt,50,200,2,20 100,9022.5,184 96 106 12 100 49 193 46 104 74 20 86 120 1...,8241.658913,...,True,0.000554,9017.5,9022.5,9022.5,Wilson,-100.0,20,100,10
95,none,mip_separation,RB2005010.txt,50,200,2,40 20,7895.5,73 27 150 60 83 96 113 177 184 48 158 148 109 ...,7475.842080,...,True,0.024392,7707.5,7895.5,7895.5,Wilson,-100.0,40,20,10
96,none,mip_separation,RB2005010.txt,50,200,2,40 40,8030.5,73 27 150 60 83 96 113 177 184 48 158 148 109 ...,8013.475852,...,True,0.056019,7604.5,8030.5,8030.5,Wilson,-100.0,40,40,10
97,none,mip_separation,RB2005010.txt,50,200,2,40 60,8350.0,73 27 150 60 83 96 113 177 184 48 158 148 109 ...,7424.404119,...,True,0.001860,8334.5,8350.0,8350.0,Wilson,-100.0,40,60,10
98,none,mip_separation,RB2005010.txt,50,200,2,40 80,8754.0,184 67 200 131 137 100 12 118 188 98 198 63 96...,7265.259320,...,True,0.000571,8749.0,8754.0,8754.0,Wilson,-100.0,40,80,10
99,none,mip_separation,RB2005010.txt,50,200,2,40 100,9022.5,184 49 195 137 128 185 133 176 145 98 61 115 1...,4136.471369,...,True,0.000000,9022.5,9022.5,9022.5,Wilson,-100.0,40,100,10


In [27]:
df[(df['cut_count'] < 0)&(df['instance_name'] == 'RB1502008.txt')]

,executionId,ub_name,instance_name,alpha,n,m,budget_Gamma,cmax,permutation,time_spent,...,validated,gap,lb,cost,cmax_dp,model,cut_count,Gamma1,Gamma2,seq
425,none,mip_separation,RB1502008.txt,20,150,2,20 20,4903.0,104 149 139 6 127 5 144 126 49 95 135 33 103 6...,346.618679,...,True,0.000000e+00,4903.000000,4903.000000,4903.0,Wilson,-100.0,20,20,08
426,none,mip_separation,RB1502008.txt,20,150,2,20 40,5135.0,124 4 110 136 17 68 116 78 5 13 49 108 83 84 1...,614.409735,...,True,0.000000e+00,5135.000000,5135.000000,5135.0,Wilson,-100.0,20,40,08
427,none,mip_separation,RB1502008.txt,20,150,2,20 60,5324.2,104 61 135 149 53 43 97 4 108 21 132 37 94 91 ...,7821.715995,...,True,1.878569e-04,5323.200000,5324.200000,5324.2,Wilson,-100.0,20,60,08
428,none,mip_separation,RB1502008.txt,20,150,2,20 80,5465.2,104 69 135 39 28 148 19 123 149 97 50 130 9 86...,730.878882,...,True,-4.992469e-16,5465.200000,5465.200000,5465.2,Wilson,-100.0,20,80,08
429,none,mip_separation,RB1502008.txt,20,150,2,20 100,5547.6,37 108 50 69 144 62 31 12 57 35 73 28 135 42 1...,394.681322,...,True,-1.639438e-16,5547.600000,5547.600000,5547.6,Wilson,-100.0,20,100,08
430,none,mip_separation,RB1502008.txt,20,150,2,40 20,4991.6,139 10 97 90 110 70 141 102 59 134 4 49 130 75...,7891.892451,...,True,1.056809e-02,4939.400000,4991.600000,4991.6,Wilson,-100.0,40,20,08
431,none,mip_separation,RB1502008.txt,20,150,2,40 40,5135.0,124 38 68 4 97 148 109 139 63 7 47 107 57 37 5...,10078.338640,...,True,2.857032e-04,5133.533333,5133.533333,5135.0,Wilson,-100.0,40,40,08
432,none,mip_separation,RB1502008.txt,20,150,2,40 60,5324.2,104 17 12 19 135 120 52 109 82 107 142 20 97 5...,330.186194,...,True,-1.708228e-16,5324.200000,5324.200000,5324.2,Wilson,-100.0,40,60,08
433,none,mip_separation,RB1502008.txt,20,150,2,40 80,5465.2,37 4 12 61 110 101 97 127 124 67 106 95 116 98...,744.303406,...,True,-4.992469e-16,5465.200000,5465.200000,5465.2,Wilson,-100.0,40,80,08
434,none,mip_separation,RB1502008.txt,20,150,2,40 100,5547.6,104 17 49 21 103 107 89 100 40 97 133 118 119 ...,220.738825,...,True,-4.918314e-16,5547.600000,5547.600000,5547.6,Wilson,-100.0,40,100,08


In [28]:
df = df[((df['cut_count'] > 0)&(df['instance_name'] == 'RB1502008.txt')&(df['model']=='Wagner')) | ((df['instance_name'] == 'RB1502008.txt')&(df['model']=='Wilson')) | (df['instance_name'] != 'RB1502008.txt')]
df = df[((df['cut_count'] > 0)&(df['instance_name'] == 'RB2005010.txt')&(df['model']=='Wagner')) | ((df['instance_name'] == 'RB2005010.txt')&(df['model']=='Wilson')) | (df['instance_name'] != 'RB2005010.txt')]

In [29]:
print(len(df.index))

15364


### Round columns containing time (in seconds) 

In [30]:
df['time_spent'] = df['time_spent'].round(2)
df['time_to_best_sol'] = df['time_to_best_sol'].round(2)

### Sort data according to model, instance_name, alpha, n, m, Gamma1 and Gamma2 and set index

In [31]:
print('Sorting dataset...')
df = df.sort_values(['model', 'n', 'm', 'alpha', 'instance_name', 'Gamma1', 'Gamma2'])
display(df.dtypes)
df = df.set_index(['model', 'n', 'm', 'alpha', 'instance_name', 'Gamma1', 'Gamma2'])
display(df.head(6))

Sorting dataset...


executionId               object
ub_name                   object
instance_name             object
alpha                      int64
n                          int64
m                          int64
budget_Gamma              object
cmax                     float64
permutation               object
time_spent               float64
time_to_best_sol         float64
iterations                 int64
num_visited_solutions      int64
num_improvements           int64
is_optimal                  bool
validated                   bool
gap                      float64
lb                       float64
cost                     float64
cmax_dp                  float64
model                     object
cut_count                float64
Gamma1                     int64
Gamma2                     int64
seq                       object
dtype: object

executionId         ub_name  \
model  n  m alpha instance_name Gamma1 Gamma2                               
Wagner 10 2 10    RB0101001.txt 20     20            none  mip_separation   
                                       40            none  mip_separation   
                                       60            none  mip_separation   
                                       80            none  mip_separation   
                                       100           none  mip_separation   
                                40     20            none  mip_separation   

                                              budget_Gamma   cmax  \
model  n  m alpha instance_name Gamma1 Gamma2                       
Wagner 10 2 10    RB0101001.txt 20     20            20 20  285.1   
                                       40            20 40  285.1   
                                       60            20 60  285.1   
                                       80            20 80  285.1   
                                       100          20 100  285.1   
                                40     20            40 20  291.8   

                                                        permutation  \
model  n  m alpha instance_name Gamma1 Gamma2                         
Wagner 10 2 10    RB0101001.txt 20     20      10 9 7 5 3 4 2 6 1 8   
                                       40      10 9 2 4 1 7 6 5 3 8   
                                       60      10 1 2 4 3 7 9 5 6 8   
                                       80      10 2 7 5 9 6 3 4 1 8   
                                       100     10 1 2 7 5 4 9 6 3 8   
                                40     20      10 9 7 5 3 4 2 6 1 8   

                                               time_spent  time_to_best_sol  \
model  n  m alpha instance_name Gamma1 Gamma2                                 
Wagner 10 2 10    RB0101001.txt 20     20            6.34              6.34   
                                       40            0.11              0.11   
                                       60            0.08              0.08   
                                       80            0.06              0.06   
                                       100           0.06              0.06   
                                40     20            0.04              0.04   

                                               iterations  \
model  n  m alpha instance_name Gamma1 Gamma2               
Wagner 10 2 10    RB0101001.txt 20     20               2   
                                       40               4   
                                       60               3   
                                       80               3   
                                       100              3   
                                40     20               2   

                                               num_visited_solutions  \
model  n  m alpha instance_name Gamma1 Gamma2                          
Wagner 10 2 10    RB0101001.txt 20     20                          2   
                                       40                          4   
                                       60                          3   
                                       80                          3   
                                       100                         3   
                                40     20                          2   

                                               num_improvements  is_optimal  \
model  n  m alpha instance_name Gamma1 Gamma2                                 
Wagner 10 2 10    RB0101001.txt 20     20                     1        True   
                                       40                     3        True   
                                       60                     2        True   
                                       80                     2        True   
                                       100                    2        True   
                                40     20              

### Find missing results, for a given value of alpha, n and m

For a given group of alpha, n, m and budget_Gamma, there should be 10 results.

First we will build a dataframe with the instances list and all required budget values.

In [32]:
data = []
rootfolder = os.getcwd()
jobs_folders = glob.glob(os.path.join(rootfolder, 'instances', 'robust', 'ying', 'data', '*/'), recursive=False)
for job_path in jobs_folders:
    alpha_folders = glob.glob(os.path.join(job_path, '*/'), recursive=False)
    n = job_path[job_path.find('data')+5:job_path.rfind('jobs')]
    #print('n: {}'.format(n))
    for alpha_path in alpha_folders:
        alpha = alpha_path[alpha_path.find('jobs')+5:alpha_path.rfind('%')]
        #print('alpha: {}'.format(alpha))
        instance_paths = glob.glob(os.path.join(alpha_path, '*'), recursive=False)
        for instance_path in instance_paths:
            instance_name = instance_path[instance_path.find('%')+2:]
            #print(instance_name)
            for gamma1 in [20, 40, 60, 80, 100]:
                for gamma2 in [20, 40, 60, 80, 100]:
                    for model in ['Wilson', 'Wagner']:
                        data.append([model, instance_name, alpha, n, 2, gamma1, gamma2])
df_instances = pd.DataFrame(data, columns=['model', 'instance_name', 'alpha', 'n', 'm', 'Gamma1', 'Gamma2'])
for col in df_instances:
    if col in ['alpha','n','m','Gamma1','Gamma2']:
        df_instances[col] = pd.to_numeric(df_instances[col], errors='coerce')
display(df_instances.dtypes)
df_instances = df_instances.set_index(['model', 'n', 'm', 'alpha', 'instance_name', 'Gamma1', 'Gamma2'])
display(df_instances)

model            object
instance_name    object
alpha             int64
n                 int64
m                 int64
Gamma1            int64
Gamma2            int64
dtype: object

,,,,,,
model,n,m,alpha,instance_name,Gamma1,Gamma2
Wilson,100,2,10,RB1001001.txt,20,20
Wagner,100,2,10,RB1001001.txt,20,20
Wilson,100,2,10,RB1001001.txt,20,40
Wagner,100,2,10,RB1001001.txt,20,40
Wilson,100,2,10,RB1001001.txt,20,60
...,...,...,...,...,...,...
Wagner,50,2,50,RB0505010.txt,100,60
Wilson,50,2,50,RB0505010.txt,100,80
Wagner,50,2,50,RB0505010.txt,100,80


Now, lets join the instances dataframe with the results one (left join).

In [33]:
df_joined = df_instances.join(df, how='left')
df_joined

executionId         ub_name  \
model  n   m alpha instance_name Gamma1 Gamma2                               
Wagner 10  2 10    RB0101001.txt 20     20            none  mip_separation   
                                        40            none  mip_separation   
                                        60            none  mip_separation   
                                        80            none  mip_separation   
                                        100           none  mip_separation   
...                                                    ...             ...   
Wilson 200 2 50    RB2005010.txt 100    20            none  mip_separation   
                                        40            none  mip_separation   
                                        60            none  mip_separation   
                                        80            none  mip_separation   
                                        100           none  mip_separation   

                                               budget_Gamma    cmax  \
model  n   m alpha instance_name Gamma1 Gamma2                        
Wagner 10  2 10    RB0101001.txt 20     20            20 20   285.1   
                                        40            20 40   285.1   
                                        60            20 60   285.1   
                                        80            20 80   285.1   
                                        100          20 100   285.1   
...                                                     ...     ...   
Wilson 200 2 50    RB2005010.txt 100    20           100 20  9192.0   
                                        40           100 40  9192.0   
                                        60           100 60  9192.0   
                                        80           100 80  9192.0   
                                        100         100 100  9192.0   

                                                                                      permutation  \
model  n   m alpha instance_name Gamma1 Gamma2                                                      
Wagner 10  2 10    RB0101001.txt 20     20                                   10 9 7 5 3 4 2 6 1 8   
                                        40                                   10 9 2 4 1 7 6 5 3 8   
                                        60                                   10 1 2 4 3 7 9 5 6 8   
                                        80                                   10 2 7 5 9 6 3 4 1 8   
                                        100                                  10 1 2 7 5 4 9 6 3 8   
...                                                                                           ...   
Wilson 200 2 50    RB2005010.txt 100    20      73 27 150 60 83 96 113 177 184 48 158 148 109 ...   
                                        40      73 27 150 60 83 96 113 177 184 48 158 148 109 ...   
                                        60      73 27 150 60 83 96 113 177 184 48 158 148 109 ...   
                                        80      73 27 150 60 83 96 113 177 184 48 158 148 109 ...   
                                        100     73 27 150 60 83 96 113 177 184 48 158 148 109 ...   

                                                time_spent  time_to_best_sol  \
model  n   m alpha instance_name Gamma1 Gamma2                                 
Wagner 10  2 10    RB0101001.txt 20     20            6.34              6.34   
                                        40            0.11              0.11   
                                        60            0.08              0.08   
                                        80            0.06              0.06   
                                        100           0.06              0.06   
...                                                    ...               ...   
Wilson 200 2 50    RB2005010.txt 100    20        11273.43          11273.43   
                                        40        10169.19          10169.

Now we will export to CSV a list with all rows with NaN values (missing experimental results).

In [34]:
missing_df = df_joined[df_joined.isnull().any(axis=1)].reset_index()[['model', 'n', 'm', 'alpha', 'instance_name', 'Gamma1', 'Gamma2']]
outputfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
if not os.path.exists(outputfolder):
    os.makedirs(outputfolder)
print('Saving file on folder: ' + outputfolder)
fname = os.path.join(outputfolder, '2RPFS_Cmax_missing_results.csv')
missing_df.to_csv(fname, sep=';')
print('Saved: ' + fname)

Saving file on folder: /public/doutorado_files/2RPFS_Cmax_Budget/results/consolidated
Saved: /public/doutorado_files/2RPFS_Cmax_Budget/results/consolidated/2RPFS_Cmax_missing_results.csv


In [35]:
df_grouped = df.groupby(['alpha', 'n', 'm', 'budget_Gamma']).agg({'executionId' : ['count']}).reset_index()
df_grouped.columns = [ ' '.join(str(i) for i in col) for col in df_grouped.columns]
#df_grouped.reset_index(inplace=True)
df_grouped

,alpha,n,m,budget_Gamma,executionId count
0,10,10,2,100 100,20
1,10,10,2,100 20,20
2,10,10,2,100 40,20
3,10,10,2,100 60,20
4,10,10,2,100 80,20
...,...,...,...,...,...
745,50,200,2,80 100,21
746,50,200,2,80 20,21
747,50,200,2,80 40,21
748,50,200,2,80 60,21


In [36]:
table = pd.pivot_table(df, values='executionId', index=['alpha', 'n'], columns=['Gamma1', 'Gamma2'], aggfunc='count', fill_value=0)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(table)

Gamma1    20                  40                  60                  80       \
Gamma2    20  40  60  80  100 20  40  60  80  100 20  40  60  80  100 20  40    
alpha n                                                                         
10    10   20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      20   20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      50   20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      100  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      150  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      200  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
20    10   20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      20   20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      50   20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      100  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      150  20  20  20  20  20  20  19  20  20  20  20  20  20  20  20  20  20   
      200  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
30    10   20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      20   20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      50   20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      100  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      150  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      200  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
40    10   20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      20   20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      50   20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      100  21  21  21  21  21  21  21  21  21  21  21  20  20  20  20  20  20   
      150  22  22  21  21  21  21  21  21  21  21  21  21  21  21  21  21  21   
      200  24  25  25  25  25  25  24  23  23  23  23  23  23  23  23  24  23   
50    10   20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      20   20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      50   20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      100  28  28  28  28  27  27  27  27  27  27  27  27  27  27  27  27  27   
      150  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20  20   
      200  24  24  24  25  24  25  25  24  23  23  24  23  23  22  22  21  21   

Gamma1                100                  
Gamma2    60  80  100 20  40  60  80  100  
alpha n                                    
10    10   20  20  20  20  20  20  20  20  
      20   20  20  20  20  20  20  20  20  
      50   20  20  20  20  20  20  20  20  
      100  20  20  20  20  20  20  20  20  
      150  20  20  20  20  20  20  20  20  
      200  20  20  20  20  20  20  20  20  
20    10   20  20  20  20  20  20  20  20  
      20   20  20  20  20  20  20  20  20  
      50   20  20  20  20  20  20  20  20  
      100  20  20  20  20  20  20  20  20  
      150  20  20  20  20  20  20  20  20  
      200  20  20  20  20  20  20  20  20  
30    10   20  20  20  20  20  20  20  20  
      20   20  20  20  20  20  20  20  20  
      50   20  20  20  20  20  20  20  20  
      100  20  20  20  20  20  20  20  20  
      150  20  20  20  20  20  20  20  20  
      200  20  20  20  20  20  20  20  20  
40    10   20  20  20  20  20  20  20  20  
      20   20  20  20  20  20  20  20  20  
      50   20  20  20  20  20  20  20  20  
      100  20  20  20  20  20  20  20  20  
      150  21  21  21  21  21  21  21  21  
      200  23  23  23  23  23  23  23  23  
50    10   20  20  20  20  20  20  20  20  
      20   20  20  20  20  20  20  20  20  
      50   20  20  20  20  20  20  20  20  
      

### Export the dataset to CSV file 

In [37]:
%%time

print('Saving file on folder: ' + outputfolder)
fname = os.path.join(outputfolder, '2RPFS_Cmax_all_results.csv')
df.to_csv(fname, sep=';')
print('Saved: ' + fname)

Saving file on folder: /public/doutorado_files/2RPFS_Cmax_Budget/results/consolidated
Saved: /public/doutorado_files/2RPFS_Cmax_Budget/results/consolidated/2RPFS_Cmax_all_results.csv
CPU times: user 762 ms, sys: 60.8 ms, total: 823 ms
Wall time: 1.81 s
